In [1]:
from regraph.neo4j.graphs import Neo4jGraph
from regraph.neo4j.cypher_utils import *

In [2]:
g = Neo4jGraph("bolt://localhost:7687", "neo4j", "admin")

In [3]:
res = g.clear()

In [4]:
nodes = ["a", "b", "c", "d", "e", "f"]
edges = [("a", "b"), ("d", "b"), ("a", "c"), ("e", "a"), ("f", "d")]
g.add_nodes_from(nodes)
g.add_edges_from(edges)

MATCH (f), (c), (e), (d), (b), (a) WHERE f.id = 'f' and c.id = 'c' and e.id = 'e' and d.id = 'd' and b.id = 'b' and a.id = 'a' CREATE (a)-[:edge]->(b), (d)-[:edge]->(b), (a)-[:edge]->(c), (e)-[:edge]->(a), (f)-[:edge]->(d)


In [5]:
g.add_node("x")

In [6]:
g.add_edge("x", "c")

MATCH (c), (x) WHERE c.id = 'c' and x.id = 'x' CREATE (x)-[:edge]->(c)


In [7]:
g.remove_edge("x", "c")

In [8]:
g.nodes()

['a', 'b', 'c', 'd', 'e', 'f', 'x']

In [9]:
g.edges()

[('e', 'a'), ('a', 'b'), ('d', 'b'), ('a', 'c'), ('f', 'd')]

In [10]:
g.clone_node('a')

In [11]:
g.clone_node('a', 'a_clone')

In [12]:
merge_nodes(["a", "b"], "a_b")

"MATCH (a:node { id : 'a' }) MATCH (b:node { id : 'b' }) OPTIONAL MATCH (a)-[:edge]->(suc_a) OPTIONAL MATCH (pred_a)-[:edge]->(a) OPTIONAL MATCH (b)-[:edge]->(suc_b) OPTIONAL MATCH (pred_b)-[:edge]->(b)  WITH a, b, COLLECT(suc_a) as sucs_a, COLLECT(pred_a) as preds_a, COLLECT(suc_b) as sucs_b, COLLECT(pred_b) as preds_b CREATE (a_b:node { id: 'a_b' }) FOREACH(s IN sucs_a | CREATE UNIQUE (a_b)-[:edge]->(s))  FOREACH(s IN sucs_b | CREATE UNIQUE (a_b)-[:edge]->(s)) FOREACH(p IN preds_a | CREATE UNIQUE (p)-[:edge]->(a_b))  FOREACH(p IN preds_b | CREATE UNIQUE (p)-[:edge]->(a_b)) FOREACH(n IN sucs_a + preds_a + sucs_b + preds_b | FOREACH(x IN CASE WHEN n IN [a, b] THEN ['dummy'] ELSE [] END |CREATE UNIQUE (a_b)-[:edge]->(a_b)) ))  DETACH DELETE a, b"